In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# import gwatpy.mcmc_routines as gmcmc
from corner import corner
import h5py 
import bayesshippy.mcmcRoutines as bmcmc


Set parameters to pick out dataset

In [ ]:
beta = 2
M = 3
sigma = 1
t = 100
dt =1
run_id = "{}_{}_{}_{}_{}".format(beta,M,sigma,t,dt)
dirname = "../data/{}/".format(run_id)

Routines for unpacking 

True data 

In [ ]:
N = 100
time = np.linspace(0,N,N)
dn = 2/ (N-1)
sigma = 1
beta = 5

True data: created from a polynomial of order M and beta and noise sigma: 
$$ y(t) =  \sum_i^M \alpha_i (t/T)^i + \mathcal{N}(0,\sigma^2)$$
with $ T $ being the length of the signal
$$
\alpha \sim \mathcal{N} (0,\beta^2)
$$


In [ ]:
true_data = np.loadtxt(dirname+"full_data_transdimensional.csv")
true_clean_data = np.loadtxt(dirname+"clean_data_transdimensional.csv")
fig, ax = plt.subplots(figsize=[8,5])
ax.plot(true_data,label='True data with noise',alpha=.8)
ax.plot(true_clean_data,label='True data without noise',alpha=.8)
ax.set_xlabel("Time")
plt.legend()
plt.savefig("../plots/transdimensional_true_data.jpg")
plt.show()
plt.close()

In [ ]:
labels = [r'$\sigma$']
for x in np.arange(10):
    labels.append('p_{}'.format(x+1))

Recovery model is Chebyshev polynomial with order P, assuming gaussian noise and fitting for $ \sigma $ of the noise. This means the model dimension is P+1
$$
y'(t) = \sum_i^P p_i \cos\left( i \times \arccos( x)\right)
$$
where $ x $ is 
$$
x = -1 + 2 t/T
$$

The likelihood is chi squared (with $N = T/dt$):
$$
\ln \mathcal{L} = - \sum_i^{N} \left( y (t_i) -  y' (t_i)\right)^2 / (2 \sigma^2 ) - (N/2)\log\left( 2 \pi \sigma^2\right) 
$$

The prior for the model is a simple flat prior for each parameter

$$
p(\sigma) \sim U(0.01,10)
$$

for the noise standard deviation, and

$$
p(p_i) \sim U(-10,10)
$$

In [ ]:
def cheb_fn(P,coeff,x ):
    return np.sum(coeff[:P] * np.cos(np.arange(P)*np.arccos(x)))

Import the RJPTMCMC data and unpack 

In [ ]:
dataDir = ""
# dataDir = "equalModelWeight/"
# dataDir = "unadjustedModelWeight/"
print("Reading in file: ",dirname+"{}transdimensionalChebyshev_output.hdf5".format(dataDir))
datafile = h5py.File(dirname+"{}transdimensionalChebyshev_output.hdf5".format(dataDir))
data, status, model_status = bmcmc.RJMCMC_unpack_file(dirname+"{}transdimensionalChebyshev_output.hdf5".format(dataDir))
data = [data,status]
datafilePrior = h5py.File(dirname+"{}transdimensionalChebyshevPrior_output.hdf5".format(dataDir))
dataPrior, statusPrior, model_statusPrior = bmcmc.RJMCMC_unpack_file(dirname+"{}transdimensionalChebyshevPrior_output.hdf5".format(dataDir))
dataPrior = [dataPrior,statusPrior]


Check some basic plots -- plot trace/histogram of one dimension at a time 

In [ ]:
ID = 0

plt.plot(data[0][np.array(data[1][:,ID],dtype=bool),ID])
plt.title("Trace for Parameter: {}".format(labels[ID]))
plt.show()
plt.close()
plt.hist(data[0][np.array(data[1][:,ID],dtype=bool),ID],bins=50)
plt.title("Hist for Parameter: {}".format(labels[ID]))
plt.show()
plt.close()

Plot Histograms of the dimensions -- probability of different models 

In [ ]:
bins = np.linspace(0.5,len(data[1][0])+.5,len(data[1][0])+1)
mids = bins[1:] - .5
print(bins)
#Collapse dimensions structure to the sum of the active dimensions (N samples, maxDim) => (N samples)
dims = np.sum(data[1],axis=1)

fig, ax = plt.subplots(figsize=[15,8])
probs, bins, _patches = ax.hist(dims,bins=bins,log=True,density=True)
print("Probabilities:")
[print("Dim: {}".format(x+1),"Probability: {:.3e}".format( probs[x])) for x in np.arange(len(probs))]
ax.set_xlabel("Dimension of the model")
ax.set_ylabel("Probability")
plt.savefig("../plots/posterior_modelspace.jpg")
plt.savefig("../plots/posterior_modelspace.pdf")
plt.show()
plt.close()


In [ ]:
plt.plot(dims)
plt.xlabel("Iteration")
plt.ylabel("Dimension")
plt.show()
plt.close()

In [ ]:
bins = np.linspace(0.5,len(data[1][0])+.5,len(data[1][0])+1)
mids = bins[1:] - .5
print(bins)
dims = np.sum(dataPrior[1],axis=1)
print(len(dims))
fig, ax = plt.subplots(figsize=[15,8])
probs, bins, _patches = ax.hist(dims,bins=bins,log=True,density=True)
print("| Dim | Probability | Scaling relative to 2 dimensions(Emperical) | Scaling (expected)")
print("| --- | --- | --- | --- |")
[print("|{}".format(x+2),"{:.3e}".format( probs[x+1])," {:.3e}".format( probs[x+1]/probs[1])," {:.3e} |".format((1./20.)**x),sep=' | ') for x in np.arange(len(probs)-1)]
# [print("| Dim: {}".format(x+2),"Probability: {:.3e}".format( probs[x+1]),"Scaling Relative to 2 Dim: {:.3e}".format( probs[x+1]/probs[1]),"Scaling expectation: {:.3e} |".format((1./20.)**x),sep=' | ') for x in np.arange(len(probs)-1)]
# [print( (1./20.)**x , cts[x+1]/cts[1], ((1./20.)**x - cts[x+1]/cts[1])/((1./20.)**x ) ) for x in np.arange(len(cts[1:])) ]

ax.set_xlabel("Dimension of the model")
ax.set_ylabel("Probability")
ax.set_ylim((10**-5,1))
plt.savefig("../plots/prior_modelspace_adjusted.jpg")
plt.savefig("../plots/prior_modelspace_adjusted.pdf")
plt.show()
plt.close()


In [ ]:
plt.plot(dims)

In [ ]:
ID = 0

plt.plot(dataPrior[0][np.array(dataPrior[1][:,ID],dtype=bool),ID])
plt.title("Trace for Parameter: {}".format(labels[ID]))
plt.show()
plt.close()
plt.hist(dataPrior[0][np.array(dataPrior[1][:,ID],dtype=bool),ID],bins=50)
plt.title("Hist for Parameter: {}".format(labels[ID]))
plt.show()
plt.close()

Used a traditional PTMCMC with the most favored model (P=5 or 6 dimensions)

In [ ]:
p = 3
dataSingle = bmcmc.MCMC_unpack_file(dirname+"transdimensionalChebyshevSingle_{}_output.hdf5".format(p))
print("data shape: ",dataSingle.shape)
fig = corner(dataSingle)
plt.show()
plt.close()


In [ ]:
fig = corner(dataSingle,weights=np.ones(len(dataSingle))/len(dataSingle),labels=labels,show_titles=True)
dims = np.sum(data[1],axis=1)
dataPlot = data[0][dims==len(dataSingle[0]),:len(dataSingle[0])]
corner(dataPlot,fig=fig,weights=np.ones(len(dataPlot))/len(dataPlot),color='green')
plt.savefig("../plots/MCMC_RJMCMC_comp.jpg")
plt.savefig("../plots/MCMC_RJMCMC_comp.pdf")
plt.show()
plt.close()

In [ ]:
ids = np.random.choice(np.arange(len(data[0])),size = 1000)
recon = [[cheb_fn(np.sum(data[1][x,1:]), data[0][x,1:],-1 + dn *t) for t in np.arange(len(time))] for x in ids]
reconPrior = [[cheb_fn(np.sum(dataPrior[1][x,1:]), dataPrior[0][x,1:],-1 + dn *t) for t in np.arange(len(time))] for x in np.random.choice(np.arange(len(dataPrior[0])),size = 1000)]
reconSingle = [[cheb_fn(len(dataSingle[0])-1, dataSingle[x,1:],-1 + dn *t) for t in np.arange(len(time))] for x in np.random.choice(np.arange(len(dataSingle)),size = 1000)]


In [ ]:
for x in recon:
    plt.plot(x,alpha=.5,linewidth=1)
plt.show()
plt.close()
for x in reconSingle:
    plt.plot(np.array(x),alpha=.5,linewidth=1)
plt.show()
plt.close()
for x in reconPrior:
    plt.plot(x,alpha=.5,linewidth=1)
plt.show()
plt.close()

Calculate 90% and mean signals 

In [ ]:
up = np.quantile(recon,.9,axis=0)
down = np.quantile(recon,.1,axis=0)
upSingle = np.quantile(reconSingle,.9,axis=0)
downSingle = np.quantile(reconSingle,.1,axis=0)
upPrior = np.quantile(reconPrior,.9,axis=0)
downPrior = np.quantile(reconPrior,.1,axis=0)
 
mean = np.mean(recon,axis=0)
meanSingle = np.mean(reconSingle,axis=0)
meanPrior = np.mean(reconPrior,axis=0)


Plot confidence regions and residuals 

In [ ]:
fig,ax = plt.subplots(nrows=2,ncols=1,sharex=True,figsize=[8,8])
ax[0].plot(time,true_data,label='Full Data')
ax[0].plot(time,true_clean_data,label="True Signal")
ax[0].fill_between(time,up,down,alpha=.8,label='RJMCMC')
ax[0].fill_between(time,upSingle,downSingle,alpha=.8,label='MCMC')
# ax.fill_between(time,upPrior,downPrior,alpha=.3,label='RJMCMC Prior')
ax[0].legend()
ax[1].plot(time,true_data-mean,alpha=.8,label='RJMCMC (mean) Residual')
ax[1].plot(time,true_data-meanSingle,alpha=.8,label='MCMC (mean) Residual')
# ax.plot(time,true_data-meanPrior,alpha=.8,label='Prior (mean) Residual')
ax[1].legend()
ax[1].set_xlabel("Time")
plt.subplots_adjust( hspace=0)
plt.savefig("../plots/reconstructions_ptrjmcmc.jpg")
plt.savefig("../plots/reconstructions_ptrjmcmc.pdf")
plt.show()
plt.close()

sigmaFit = np.mean(dataSingle[:,0])
print("Likelihood approx for single dim: ", np.sum( -(true_data-meanSingle)**2/(2.*sigmaFit*sigmaFit) - (len(time)/2) * np.log(2.*np.pi * sigmaFit*sigmaFit)) )
sigmaFit = np.mean(data[0][:,0])
print("Likelihood approx for RJ: ", np.sum( -(true_data-mean)**2/(2.*sigmaFit*sigmaFit) - (len(time)/2) * np.log(2.*np.pi * sigmaFit*sigmaFit)) )



Load in Bilby evidences 

In [ ]:
lnE = []
# mods = np.array([1,2,3,4,5,6,7,8,9])
mods = np.array([1,2,3,4,5,6,7,8])
for x in mods:
#     lnE.append(np.loadtxt("../data/bilby_transdimensionallog_evidence_{}.txt".format(x)))
    lne = np.loadtxt(dirname+"bilby/log_evidence_{}.txt".format(x))
    
    lnE.append(float(lne))
# print(lnE)
evidence = np.exp(np.array(lnE)) / np.sum(np.exp(np.array(lnE)))
# print(evidence)
fig, ax = plt.subplots()
ax.scatter(mods+1,evidence)
ax.set_yscale('log')
ax.set_ylim([10**-5,1.1])
ax.set_ylabel("Probability")
ax.set_xlabel("Dimension")
plt.show()
plt.close()

In [ ]:
bins = np.linspace(0.5,len(data[1][0])+.5,len(data[1][0])+1)
mids = bins[1:] - .5
# print(bins)
dims = np.sum(data[1],axis=1)

fig, ax = plt.subplots(figsize=[8,5])
# ax.hist(dims,bins=bins,density=True,alpha=.8,label='RJPTMCMC')
ax.hist(dims,bins=bins,log=True,density=True,alpha=.8,label='RJPTMCMC')
ax.scatter(mods+1,evidence,color='red',label='Bilby')
ax.set_ylim((10**-4,1.1))
ax.set_xlabel("Dimension")
ax.legend()
plt.savefig("../plots/evidence_bilby_vs_ptrjmcmc_adjusted.jpg")
plt.savefig("../plots/evidence_bilby_vs_ptrjmcmc_adjusted.pdf")
plt.show()
plt.close()


Load in Bilby Samples 

In [ ]:
dataBilby = []
for x in np.arange(8):
    dataBilby.append(np.loadtxt(dirname+"bilby/{}_samples.dat".format(x+1),skiprows=1))
    print(dataBilby[-1].shape)    

Reconstruct the signal with Bilby samples 

In [ ]:
reconBilby = [[[cheb_fn(ct+1,dataBilby[ct][x,1:],-1 + dn *t) for t in np.arange(len(time))] for x in np.random.choice(np.arange(len(dataBilby[ct])),size = 1000)] for ct in np.arange(len(dataBilby))]


In [ ]:
ct = 2
for d in reconBilby:
    fig,ax=plt.subplots(nrows=2,ncols=1,sharex=True,figsize=[8,8])
    
#     for x in d:
#         ax.plot(x,alpha=.5,linewidth=1)
    upB = np.quantile(d,.9,axis=0)
    downB = np.quantile(d,.1,axis=0)
    meanB = np.mean(d,axis=0)
    ax[0].fill_between(time,upB,downB,alpha=.3,label='Dynesty')
    ax[0].fill_between(time,up,down,alpha=.3,label='RJMCMC',color='r')

    ax[0].plot(time,true_data,label='Full Data')
    ax[0].plot(time,true_clean_data,label="True Signal")
    ax[0].set_title("Dimensions: {}".format(ct))
    ax[0].legend()
    
    ax[1].plot(time,true_data-meanB,alpha=.8,label='Dynesty (mean) Residual')
    ax[1].plot(time,true_data-mean,alpha=.8,label='RJMCMC (mean) Residual')
    ax[1].plot(time,true_data-meanSingle,alpha=.8,label='MCMC (mean) Residual')
    ax[1].legend()

    ax[1].set_xlabel("Time")
    plt.subplots_adjust( hspace=0)
    
    plt.savefig("../plots/reconstruction_bilby_{}.jpg".format(ct))
    plt.savefig("../plots/reconstruction_bilby_{}.pdf".format(ct))
    ct+=1
    plt.show()
    plt.close()

Focus on specific dimension comparisons 

In [ ]:
ct=2
ID = 0
startingData = 0
if ID == 0 or ID == 1:
    ct = 2
    startingData = 0
else:
    ct = 2 + (ID-1)
    startingData = ID -1
# for d in dataBilby[ID-1:]:
for d in dataBilby[startingData:]:
    print(d.shape,ID,startingData)
    fig,ax=plt.subplots(figsize=[10,5])
#     for x in d:
#         ax.plot(x,alpha=.5,linewidth=1)
   
    ax.hist(data[0][np.array(data[1][:,ID],dtype=bool),ID],bins=50,density=True,label='PTRJMCMC')
    ax.hist(d[:,ID],bins=50,density=True,label='DYNESTY')
    ax.set_title("Dimensions: {}".format(ct ))
    ax.legend()
    ct+=1
    plt.show()
    plt.close()

Corner comparison between Bilby 5 dimension and MCMC 5 dimension (Should be identical.. )

In [ ]:
bilbyID = len(dataSingle[0])-2
fig = corner(dataBilby[bilbyID],bins=30,labels=labels,show_titles=True,weights=np.ones(len(dataBilby[bilbyID]))/len(dataBilby[bilbyID]) )
corner(dataSingle,fig=fig,bins=30,labels=labels,show_titles=True,color='blue',weights=np.ones(len(dataSingle))/len(dataSingle) ) 
plt.savefig("../plots/bilby_ptmcmc_comp_5dim.pdf")
plt.savefig("../plots/bilby_ptmcmc_comp_5dim.jpg")
plt.show()
plt.close()
for x in np.arange(len(dataSingle[0])):
    cP = np.quantile(dataSingle[:,x],.95)-np.quantile(dataSingle[:,x],.05)
    cB =np.quantile(dataBilby[bilbyID][:,x],.95)-np.quantile(dataBilby[bilbyID][:,x],.05)
    print(x,cP, cB, 2*(cP - cB)/(cP + cB))    
    

Comparison of RJPTMCMC and PTMCMC 

In [ ]:
gmcmc.RJcorner(data[0][:,:5],data[1][:,:5],figsize=[15,15],titles = labels,show_quantiles=True)
plt.show()
plt.close()
gmcmc.RJcorner(dataSingle,np.ones(len(dataSingle)*5).reshape((len(dataSingle),5)),figsize=[15,15],titles = labels,show_quantiles=True)
plt.show()
plt.close()

All corners for Bilby

In [ ]:

for d in dataBilby:
    corner(d,bins=50,labels=labels,show_titles=True)
   
    plt.show()
    plt.close()

OLD -- GWATPY version with thermodynamic integration

ID = 4
plt.hist(data[0][data[1][:,ID] == 1,ID])

In [ ]:
dims = np.sum(data[1],axis=1)

In [ ]:
from scipy.integrate import trapz,quad
from scipy.interpolate import interp1d
Ts= [T1,T2,T3,T4]
LLs = [IL1,IL2,IL3,IL4]
lnE = []
for ct in np.arange(len([IL1,IL2,IL3,IL4])):
    
    betas = np.flip(1/Ts[ct][:len(LLs[ct])-1])
    LL = np.flip(LLs[ct][:-1])
    func1 = interp1d(betas,LL,kind='cubic')
    evidence = quad(func1, betas[0],betas[-1])[0]
    evidencetrapz = trapz(LL,x=betas)
    print(evidencetrapz)
    lnE.append(evidence)
print(lnE)
print(evidence1,evidence2,evidence3,evidence4)

In [ ]:
plt.plot(mids[2:6],[lnE[3],lnE[2],lnE[0],lnE[1]])

In [ ]:
print("Bayes factor 5/6 (RJ): ",np.sum(dims == 5) / np.sum(dims==6))
print("Bayes factor 5/6 (PTMCMC): ",np.exp(lnE[0] - lnE[1]))
print("Bayes factor 5/4 (RJ): ",np.sum(dims == 5) / np.sum(dims==4))
print("Bayes factor 5/4 (PTMCMC): ",np.exp(lnE[0] - lnE[2]))
print("Bayes factor 5/3 (RJ): ",np.sum(dims == 5) / np.sum(dims==3))
print("Bayes factor 5/3 (PTMCMC): ",np.exp(lnE[0] - lnE[3]))
print("Bayes factor 4/3 (RJ): ",np.sum(dims == 4) / np.sum(dims==3))
print("Bayes factor 4/3 (PTMCMC): ",np.exp(lnE[2] -lnE[3]))

In [ ]:
plt.scatter(1/T4[:len(IL4[:-1])],IL4[:-1])
print(1/T1[:len(IL1[:])])
print(IL1)
#plt.plot(IL2[:-1])
#plt.plot(IL3[:-1])
#plt.plot(IL4[:-1])

In [ ]:
print(ILNUM1)
print(ILNUM2)
print(ILNUM3)
print(ILNUM4)
print(T1)

In [ ]:
f1 = h5py.File("../data/output_trans_fixed_dim1.hdf5",'r')
LLLP1 = f1["MCMC_OUTPUT"]["LOGL_LOGP"]
num =0
ave = 0
ensemble_member = 9
ensemble_size = 10
for c in LLLP1.keys():
    chain = int(c[5:])
    #if chain%ensemble_size == ensemble_member:
    if True:
        print(chain)
        ave+= np.sum(LLLP1[c][:,0])
        num+=len(LLLP1[c][:,0])
        plt.plot(LLLP1[c][:,0])
print(ave/num)